# Document Loading

## Retrieval augmented generation
 
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution. 

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc). 

<img width="1047" alt="Screenshot 2024-04-15 at 12 24 00 PM" src="https://github.com/harrywang/langchain-short-course/assets/595772/0c3cdde0-831d-4e11-91f2-670abc580d89">

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Load PDF
Each page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`.

In [2]:
# load pdf
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/nba-rules-2023.pdf")
pages = loader.load()

In [3]:
len(pages)

74

In [4]:
# content people don't see
page = pages[0]
page.page_content


'This Page Intentionally Left Blank  \nIt is here to hold a place for cover for screen version.  \nDO NOT INCLUDE AS PART OF PRINT FILE!\nOFFICIAL\nRULES'

In [5]:
page = pages[1]
page.page_content

'- 2 -RULES INDEX\n RULE SECTION ARTICLE PAGE\nBACKCOURT/FRONTCOURT \n\t Definitions  .......................................  4 VI a & b 19\n Eight (8) Second Violation  ..............  4 VI f 19\n Player Position Status  ......................  4 VI c 19\n Ball Position Status  ..........................  4 VI d, e, & g 19\nBALL\n Dead Ball  .........................................  6 IV a 26\n Jump—Center Circle  .......................  6 V  26\n Jump—Free Throw Circle  ...............  6 VI  27\n Live Ball  ..........................................  6 II  26\n Putting in Play  .................................  6 I  25\n Restrictions  ......................................  6 VII  27\n Starting of Games & Overtime(s)  .... 6 I a 25\n Starting of 2nd, 3rd, & 4th Periods  .. 6 I b 25\nBASKET RING, BACKBOARD, SUPPORT\n\t Definition  .........................................  4 I  17\n Hanging (Intentional) .......................  12A  IV  40\n Hanging (Prevent Injury)  ...........

In [6]:
page.metadata

{'source': 'data/nba-rules-2023.pdf', 'page': 1}

In [7]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://harrywang.me/stack")
docs = loader.load()
print(docs[0].page_content[:500])













The Tech Stack that Works for My (AI) Startups | harrywang.me



















harrywang.me






























Open menu










                    Home
                



                        About
                    


                        Blog
                    





February 26, 2024
The Tech Stack that Works for My (AI) Startups



startup


tech


entrepreneurship



Last updated in Feb. 2024
In this post, I list the tech stack and related services tha


# Document Splitter


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [9]:
chunk_size =26
chunk_overlap = 3

In [10]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [11]:
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [12]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'xyzabcdefghijklmnopqrstuvw', 'uvwxyz']

In [13]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'm n o p q r s t u v w x y', 'y z']

In [14]:
c_splitter.split_text(text3)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [15]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [16]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [17]:
len(some_text)

496

In [18]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [19]:
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [20]:
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [21]:
#Let's reduce the chunk size a bit and add a period to our separators:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [22]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [23]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/nba-rules-2023.pdf")
pages = loader.load()

In [24]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)
docs = text_splitter.split_documents(pages)

In [25]:
len(docs), len(pages)

(276, 74)

## Token splitting

We can also split on token count explicity, if we want.

This can be useful because LLMs often have context windows designated in tokens.

Tokens are often ~4 characters.


In [26]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

text1 = "foo bar bazzyfoo"
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [27]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(pages)
print(docs[0], pages[0].metadata)

page_content='This Page Intentionally Left Blank  \n' metadata={'source': 'data/nba-rules-2023.pdf', 'page': 0} {'source': 'data/nba-rules-2023.pdf', 'page': 0}


In [28]:
# embeddings
# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
# By default, the length of the embedding vector will be 1536 for text-embedding-3-small or 
# 3072 for text-embedding-3-large. 
# text-embedding-3-small	$0.02 / 1M tokens
# text-embedding-3-large	$0.13 / 1M tokens

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [29]:
sentence1 = "i like dogs"
sentence2 = "i like cats"
sentence3 = "the weather is sweet outside"

In [30]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [31]:
import numpy as np
np.dot(embedding1, embedding2), np.dot(embedding1, embedding3), np.dot(embedding2, embedding3)

(0.7222818700424505, 0.2678379503528554, 0.2573718290611007)

In [32]:
!rm -rf ./data/chroma  # remove old database files if any

In [33]:
from langchain.vectorstores import Chroma
persist_directory = './data/chroma/'

In [34]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("data/getting-real/getting-real-01-introduction.pdf"),
    PyPDFLoader("data/getting-real/getting-real-02-starting-line.pdf"),
    PyPDFLoader("data/getting-real/getting-real-03-stay-lean.pdf"),  # Duplicate documents on purpose
    PyPDFLoader("data/getting-real/getting-real-03-stay-lean.pdf"),  # Duplicate documents on purpose
    PyPDFLoader("data/getting-real/getting-real-04-priorities.pdf"), 
    PyPDFLoader("data/getting-real/getting-real-05-feature-selection.pdf"), 
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

len(splits)

79

In [35]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [36]:
print(vectordb._collection.count())

79


In [37]:
question = "what's a foul in basketball?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

duplicate chunks (because of the duplicate pdfs in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

`docs[0]` and `docs[1]` are indentical.

In [38]:
docs[0].page_content

'source of emergence. In the flocking birds example, as with a well-designed system, it’s the\ninteractions and relationships that create the interesting beh avior.\nThe harder we tighten things down, the less room there is for a creative, emergent solution.\nWhether it’s locking down requirements before they are well understood or prematurely\noptimizing code, or inventing complex navigation and workflow scenarios before letting end\nusers play with the system, the result is the same: an overly complicated, stupid system instead\nof a clean, elegant system that harnesses emergence.\nKeep it small. Keep it simple. Let it happen.\n—Andrew Hunt, The Pragmatic Programmers'

In [39]:
docs[1].page_content

'source of emergence. In the flocking birds example, as with a well-designed system, it’s the\ninteractions and relationships that create the interesting beh avior.\nThe harder we tighten things down, the less room there is for a creative, emergent solution.\nWhether it’s locking down requirements before they are well understood or prematurely\noptimizing code, or inventing complex navigation and workflow scenarios before letting end\nusers play with the system, the result is the same: an overly complicated, stupid system instead\nof a clean, elegant system that harnesses emergence.\nKeep it small. Keep it simple. Let it happen.\n—Andrew Hunt, The Pragmatic Programmers'

In [40]:
docs[2].page_content

'strokes than to be wishy-washy about it. If that comes off as cocky or arrogant, so be it. We’d\nrather be provocative than water everything down with “it depends…” Of course there will be\ntimes when these rules need to be stretched or broken. And some of these tactics may not apply\nto your situation. Use your judgement and imagination.\n“This won’t work inside my company.”\nThink you’re too big to Get Real? Even Microsoft is Getting Real (and we doubt you’re bigger than\nthem).'

In [41]:
# Let's save this so we can use it later!
vectordb.persist()

/Users/harrywang/sandbox/tutorial-buffet/langchain/langchain-rag-basics/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


We can see a new failure mode.

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [42]:
question = "what did they say about regression in the third lecture?"
docs = vectordb.similarity_search(question, k=5)
for doc in docs:
    print(doc.metadata)


{'page': 2, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}
{'page': 2, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}
{'page': 4, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}
{'page': 4, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}
{'page': 3, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}


In [43]:
print(docs[4].page_content)

source of emergence. In the flocking birds example, as with a well-designed system, it’s the
interactions and relationships that create the interesting beh avior.
The harder we tighten things down, the less room there is for a creative, emergent solution.
Whether it’s locking down requirements before they are well understood or prematurely
optimizing code, or inventing complex navigation and workflow scenarios before letting end
users play with the system, the result is the same: an overly complicated, stupid system instead
of a clean, elegant system that harnesses emergence.
Keep it small. Keep it simple. Let it happen.
—Andrew Hunt, The Pragmatic Programmers


In [44]:
# Chroma in-memory
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


embedding = OpenAIEmbeddings(model="text-embedding-3-small")

persist_directory = './data/chroma/'

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

texts = [
    """Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry""",
    """Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry""",
    """Five Forces include: Competitive Rivalry, Supplier Power, Buyer Power, Threat of Substitution, and Threat of New Entry.""",
    """The Four C Model of Creativity categorizes creative expression into four levels: mini-c (personal creative learning), little-c (creativity appreciated by others), Pro-c (professional-level creativity), and Big-C (historically significant creativity). This model emphasizes creativity as a lifelong process, important at all stages of personal and professional development.""",
]

smalldb = Chroma.from_texts(texts, embedding=embedding)

print(smalldb._collection.count())  # 0 nothing loaded yet


4


In [45]:
question = "Tell me about porter's five forces model"

smalldb.similarity_search(question, k=3)

[Document(page_content="Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry"),
 Document(page_content="Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry"),
 Document(page_content='Five Forces include: Competitive Rivalry, Supplier Power, Buyer Power, Threat of Substitution, and Threat of New Entry.')]

### Addressing Diversity: Maximum marginal relevance

In [46]:
# duplicate is gone

smalldb.max_marginal_relevance_search(question, k=3, fetch_k=4)

[Document(page_content="Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry"),
 Document(page_content='Five Forces include: Competitive Rivalry, Supplier Power, Buyer Power, Threat of Substitution, and Threat of New Entry.'),
 Document(page_content='The Four C Model of Creativity categorizes creative expression into four levels: mini-c (personal creative learning), little-c (creativity appreciated by others), Pro-c (professional-level creativity), and Big-C (historically significant creativity). This model emphasizes creativity as a lifelong process, important at all stages of personal and professional development.')]

In [47]:
question = "Tell me about creativity models"
smalldb.similarity_search(question, k=2)

[Document(page_content='The Four C Model of Creativity categorizes creative expression into four levels: mini-c (personal creative learning), little-c (creativity appreciated by others), Pro-c (professional-level creativity), and Big-C (historically significant creativity). This model emphasizes creativity as a lifelong process, important at all stages of personal and professional development.'),
 Document(page_content="Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry")]

In [48]:
# cosine distance, short the better
smalldb.similarity_search_with_score(question, k=2)

[(Document(page_content='The Four C Model of Creativity categorizes creative expression into four levels: mini-c (personal creative learning), little-c (creativity appreciated by others), Pro-c (professional-level creativity), and Big-C (historically significant creativity). This model emphasizes creativity as a lifelong process, important at all stages of personal and professional development.'),
  0.6910927295684814),
 (Document(page_content="Michael Porter's five-force strategic analysis model, introduced in a 1979 article published in the Harvard Business Review, remains a fundamental tool for strategic analysts plotting the competitive landscape of an industry"),
  1.3747798204421997)]

### Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.